In [53]:
import pandas as pd
import numpy as np

In [54]:
data_cols = ['user id','movie id','rating','timestamp']
data = pd.read_csv('/workspaces/LLM-Recommender-System/data/ml-100k/u.data', sep='\t', header=None, names=data_cols)
data.head()

,user id,movie id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [55]:
item_cols = ['movie id',  'movie title', 'release date', 'video release date','IMDb URL', 'unknown',\
'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama','Fantasy', \
'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance','Sci-Fi', 'Thriller', 'War', 'Western']
item = pd.read_csv('/workspaces/LLM-Recommender-System/data/ml-100k/u.item',  sep='|',header=None,names=item_cols,encoding='latin-1')
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [56]:
# Assuming 'df' is your DataFrame
genre_columns = ['Action', 'Adventure', 'Animation', 'Children', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

# Function to get genres for a row
def get_genres(row):
    genres = [genre for genre in genre_columns if row[genre] == 1]
    return ', '.join(genres) if genres else 'unknown'

# Apply the function to each row
item['Genres'] = item.apply(get_genres, axis=1)
item.head()

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,"Animation, Children"
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,1,0,0,"Action, Adventure, Thriller"
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,Action
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,Thriller


In [57]:
item = item[['movie id', 'movie title', 'Genres']]
item.head()

,movie id,movie title,Genres
0,1,Toy Story (1995),"Animation, Children"
1,2,GoldenEye (1995),"Action, Adventure, Thriller"
2,3,Four Rooms (1995),Thriller
3,4,Get Shorty (1995),Action
4,5,Copycat (1995),Thriller


In [58]:
# merge data and item
df = pd.merge(data, item, on='movie id').sort_values(by=['user id', 'movie id']).reset_index(drop=True)
df.head()

,user id,movie id,rating,timestamp,movie title,Genres
0,1,1,5,874965758,Toy Story (1995),"Animation, Children"
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller"
2,1,3,4,878542960,Four Rooms (1995),Thriller
3,1,4,3,876893119,Get Shorty (1995),Action
4,1,5,3,889751712,Copycat (1995),Thriller


In [59]:
# average rating for each movie for each user, add the column to df
df['avg_rating'] = df.groupby(['user id', 'movie id'])['rating'].transform('mean')
# rename the column
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp', 'movie_title', 'genres', 'avg_rating']
df.head()

,user_id,movie_id,rating,timestamp,movie_title,genres,avg_rating
0,1,1,5,874965758,Toy Story (1995),"Animation, Children",5.0
1,1,2,3,876893171,GoldenEye (1995),"Action, Adventure, Thriller",3.0
2,1,3,4,878542960,Four Rooms (1995),Thriller,4.0
3,1,4,3,876893119,Get Shorty (1995),Action,3.0
4,1,5,3,889751712,Copycat (1995),Thriller,3.0


In [60]:
df.to_csv('/workspaces/LLM-Recommender-System/data/processed_movie100k.csv', index=False)

In [215]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Function to find most similar users
def get_similar_users(user_id, matrix, m):
    if user_id not in matrix.index:
        return []
    sim_users = matrix.loc[user_id].sort_values(ascending=False).iloc[1:m+1].index.tolist()
    return sim_users

def user_filtering_recommendations(dataframe, target_user_id, m, ns):
    """
    Generate movie recommendations for a target user based on user-filtering.

    :param dataframe: A pandas DataFrame containing columns 'user_id', 'movie_id', 'rating', 'movie title'.
    :param target_user_id: The user ID for whom recommendations are to be generated.
    :param m: The number of similar users to consider.
    :param ns: The number of candidate items to recommend.
    :return: A list of recommended movie titles.
    """
    # Create a pivot table
    user_movie_matrix = dataframe.pivot_table(index='user_id', columns='movie_id', values='avg_rating', fill_value=0)

    # Convert to sparse matrix
    sparse_matrix = csr_matrix(user_movie_matrix)

    # Compute Cosine Similarity
    cosine_sim = cosine_similarity(sparse_matrix)

    # Convert to DataFrame
    cosine_sim_df = pd.DataFrame(cosine_sim, index=user_movie_matrix.index, columns=user_movie_matrix.index)

    # Find similar users
    similar_users = get_similar_users(target_user_id, cosine_sim_df, m)

    # Get candidate movie IDs
    candidate_ids = dataframe[dataframe['user_id'].isin(similar_users)]['movie_id'].value_counts().head(ns).index

    # Map IDs to Titles
    candidate_titles = dataframe[dataframe['movie_id'].isin(candidate_ids)]['movie_title'].unique().tolist()

    return candidate_titles

candidate_movies = user_filtering_recommendations(df, target_user_id=15, m=10, ns=20)
print("Candidate Movies:", candidate_movies)


Candidate Movies: ['Toy Story (1995)', "Mr. Holland's Opus (1995)", 'Birdcage, The (1996)', 'Star Wars (1977)', 'Fargo (1996)', 'Independence Day (ID4) (1996)', 'Return of the Jedi (1983)', 'Jerry Maguire (1996)', "Devil's Own, The (1997)", "My Best Friend's Wedding (1997)", 'Men in Black (1997)', 'Contact (1997)', 'Full Monty, The (1997)', 'River Wild, The (1994)', 'Time to Kill, A (1996)', 'Tin Cup (1996)', 'English Patient, The (1996)', 'Air Force One (1997)', 'Game, The (1997)', 'Courage Under Fire (1996)']


In [216]:
# how many candidate movies are in the user's watch history
user_2_movies = df[df['user_id'] == 15]['movie_title'].unique().tolist()
print("User 2 Movies:", user_2_movies)
print("Number of Candidate Movies in User 2's Watch History:", len(set(candidate_movies).intersection(set(user_2_movies))))

User 2 Movies: ['Toy Story (1995)', 'Twelve Monkeys (1995)', 'Dead Man Walking (1995)', 'Mighty Aphrodite (1995)', 'Postino, Il (1994)', "Mr. Holland's Opus (1995)", 'White Balloon, The (1995)', 'Angels and Insects (1995)', 'Birdcage, The (1996)', 'Star Wars (1977)', 'Truth About Cats & Dogs, The (1996)', 'Twister (1996)', 'Independence Day (ID4) (1996)', 'Phenomenon (1996)', 'Godfather, The (1972)', 'Big Night (1996)', 'Ghost and the Darkness, The (1996)', 'Return of the Jedi (1983)', 'Mirror Has Two Faces, The (1996)', 'Star Trek: First Contact (1996)', '101 Dalmatians (1996)', 'Mars Attacks! (1996)', 'Jerry Maguire (1996)', 'Jungle2Jungle (1997)', "Smilla's Sense of Snow (1997)", 'Grosse Pointe Blank (1997)', 'Austin Powers: International Man of Mystery (1997)', 'Shall We Dance? (1996)', 'Lost World: Jurassic Park, The (1997)', "My Best Friend's Wedding (1997)", 'Men in Black (1997)', 'Contact (1997)', 'Full Monty, The (1997)', 'Sabrina (1995)', 'Sense and Sensibility (1995)', 'Bed 

In [62]:
import openai

In [185]:
temp_1 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Their genres are: {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: 
"""

temp_2 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Their genres are: {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: {}.
Step 2: You will select 5 movies that appeal to me the most from the list of movies I have watched, based on my personal preferences. The selected movies will be presented in descending order of preference. (Format: [no. a watched movie.]). 
Answer: 
"""

temp_3 = """
Candidate Set (candidate movies): {}.
The movies I have watched (watched movies): {}.
Their genres are: {}.
Step 1: What features are most important to me when selecting movies (Summarize my preferences briefly)? 
Answer: {}.
Step 2: You will select 5 movies that appeal to me the most from the list of movies I have watched, based on my personal preferences. The selected movies will be presented in descending order of preference. (Format: [no. a watched movie.]). 
Answer: {}.
Step 3: Can you recommend 10 movies from the Candidate Set similar to but not in the selected movies I've watched?.
 (Format: Here are the 10 movies recommended for you: [no. a candidate movie])
Answer: 
"""

In [186]:
watched_movies = df[df['user_id'] == 10]['movie_title'].unique().tolist()
# select random 10 movies from watched movies
np.random.seed(42)   # set random seed
selected_watched_movies = np.random.choice(watched_movies, min(len(watched_movies), 10), replace=False).tolist()
print(selected_watched_movies)
# combine movie name and genre
selected_watched_movies_genres = set(df[df['movie_title'].isin(selected_watched_movies)]['genres'].unique().tolist())
print(selected_watched_movies_genres)

['Forrest Gump (1994)', 'Top Gun (1986)', 'Chinatown (1974)', 'Philadelphia Story, The (1940)', 'All About Eve (1950)', 'Pulp Fiction (1994)', 'Silence of the Lambs, The (1991)', 'When Harry Met Sally... (1989)', 'Sabrina (1954)', 'Cinderella (1950)']
{'Thriller', 'Romance, War', 'Action, Romance', 'Animation, Children, Musical', 'Romance', 'Film-Noir, Mystery, Thriller', 'unknown'}


In [187]:
candidate_movies = user_filtering_recommendations(df, target_user_id=2, m=10, ns=20)
Input_1 = temp_1.format(candidate_movies, selected_watched_movies, selected_watched_movies_genres)
Input_1

'\nCandidate Set (candidate movies): [\'Twelve Monkeys (1995)\', \'Dead Man Walking (1995)\', \'Star Wars (1977)\', \'Fargo (1996)\', \'Godfather, The (1972)\', \'Return of the Jedi (1983)\', \'Jerry Maguire (1996)\', "My Best Friend\'s Wedding (1997)", \'Men in Black (1997)\', \'Contact (1997)\', \'Full Monty, The (1997)\', \'Sense and Sensibility (1995)\', \'Secrets & Lies (1996)\', \'English Patient, The (1996)\', \'Evita (1996)\', "Ulee\'s Gold (1997)", \'Air Force One (1997)\', \'L.A. Confidential (1997)\', \'Game, The (1997)\', \'Michael Collins (1996)\'].\nThe movies I have watched (watched movies): [\'Forrest Gump (1994)\', \'Top Gun (1986)\', \'Chinatown (1974)\', \'Philadelphia Story, The (1940)\', \'All About Eve (1950)\', \'Pulp Fiction (1994)\', \'Silence of the Lambs, The (1991)\', \'When Harry Met Sally... (1989)\', \'Sabrina (1954)\', \'Cinderella (1950)\'].\nTheir genres are: {\'Thriller\', \'Romance, War\', \'Action, Romance\', \'Animation, Children, Musical\', \'Roma

In [188]:
import openai
from key import OPENAI_API_KEY

In [189]:
openai.api_key = OPENAI_API_KEY

In [190]:
# get response from OpenAI
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= [{ 'role':'user','content' : Input_1}],
    # temperature=0.5,
    # max_tokens=512,
    # top_p=1,
    # frequency_penalty=0,
    # presence_penalty=0.6,
    # stop=["\n"]
    )

In [191]:
response

<OpenAIObject chat.completion id=chatcmpl-8RtpQ6ZAwc8qXIHVcZwdydDYIPjlf at 0x7f962022b7e0> JSON: {
  "id": "chatcmpl-8RtpQ6ZAwc8qXIHVcZwdydDYIPjlf",
  "object": "chat.completion",
  "created": 1701660152,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Based on the given information, the most important features to you when selecting movies are romance, war, action, animation, children, musical, film-noir, mystery, thriller, and unknown genres."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 351,
    "completion_tokens": 40,
    "total_tokens": 391
  },
  "system_fingerprint": null
}

In [192]:
prediction_1 = response.choices[0].message.content

In [193]:
Input_2 = temp_2.format(candidate_movies, selected_watched_movies, selected_watched_movies_genres, prediction_1)
Input_2

'\nCandidate Set (candidate movies): [\'Twelve Monkeys (1995)\', \'Dead Man Walking (1995)\', \'Star Wars (1977)\', \'Fargo (1996)\', \'Godfather, The (1972)\', \'Return of the Jedi (1983)\', \'Jerry Maguire (1996)\', "My Best Friend\'s Wedding (1997)", \'Men in Black (1997)\', \'Contact (1997)\', \'Full Monty, The (1997)\', \'Sense and Sensibility (1995)\', \'Secrets & Lies (1996)\', \'English Patient, The (1996)\', \'Evita (1996)\', "Ulee\'s Gold (1997)", \'Air Force One (1997)\', \'L.A. Confidential (1997)\', \'Game, The (1997)\', \'Michael Collins (1996)\'].\nThe movies I have watched (watched movies): [\'Forrest Gump (1994)\', \'Top Gun (1986)\', \'Chinatown (1974)\', \'Philadelphia Story, The (1940)\', \'All About Eve (1950)\', \'Pulp Fiction (1994)\', \'Silence of the Lambs, The (1991)\', \'When Harry Met Sally... (1989)\', \'Sabrina (1954)\', \'Cinderella (1950)\'].\nTheir genres are: {\'Thriller\', \'Romance, War\', \'Action, Romance\', \'Animation, Children, Musical\', \'Roma

In [194]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= [{ 'role':'user','content' : Input_2}],
    # temperature=0.5,
    # max_tokens=512,
    # top_p=1,
    # frequency_penalty=0,
    # presence_penalty=0.6,
    )

In [195]:
response

<OpenAIObject chat.completion id=chatcmpl-8RtpWnZQQdcAJTGwqO0ddtHJ2Jiey at 0x7f9620275440> JSON: {
  "id": "chatcmpl-8RtpWnZQQdcAJTGwqO0ddtHJ2Jiey",
  "object": "chat.completion",
  "created": 1701660158,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "1. Pulp Fiction (1994)\n2. Silence of the Lambs, The (1991)\n3. Sabrina (1954)\n4. Cinderella (1950)\n5. Philadelphia Story, The (1940)"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 448,
    "completion_tokens": 48,
    "total_tokens": 496
  },
  "system_fingerprint": null
}

In [196]:
prediction_2 = response.choices[0].message.content
prediction_2

'1. Pulp Fiction (1994)\n2. Silence of the Lambs, The (1991)\n3. Sabrina (1954)\n4. Cinderella (1950)\n5. Philadelphia Story, The (1940)'

In [197]:
Input_3 = temp_3.format(candidate_movies, selected_watched_movies, selected_watched_movies_genres, prediction_1, prediction_2)
Input_3

'\nCandidate Set (candidate movies): [\'Twelve Monkeys (1995)\', \'Dead Man Walking (1995)\', \'Star Wars (1977)\', \'Fargo (1996)\', \'Godfather, The (1972)\', \'Return of the Jedi (1983)\', \'Jerry Maguire (1996)\', "My Best Friend\'s Wedding (1997)", \'Men in Black (1997)\', \'Contact (1997)\', \'Full Monty, The (1997)\', \'Sense and Sensibility (1995)\', \'Secrets & Lies (1996)\', \'English Patient, The (1996)\', \'Evita (1996)\', "Ulee\'s Gold (1997)", \'Air Force One (1997)\', \'L.A. Confidential (1997)\', \'Game, The (1997)\', \'Michael Collins (1996)\'].\nThe movies I have watched (watched movies): [\'Forrest Gump (1994)\', \'Top Gun (1986)\', \'Chinatown (1974)\', \'Philadelphia Story, The (1940)\', \'All About Eve (1950)\', \'Pulp Fiction (1994)\', \'Silence of the Lambs, The (1991)\', \'When Harry Met Sally... (1989)\', \'Sabrina (1954)\', \'Cinderella (1950)\'].\nTheir genres are: {\'Thriller\', \'Romance, War\', \'Action, Romance\', \'Animation, Children, Musical\', \'Roma

In [198]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= [{ 'role':'user','content' : Input_3}],
    # temperature=0.5,
    # max_tokens=512,
    # top_p=1,
    # frequency_penalty=0,
    # presence_penalty=0.6,
    )

In [199]:
response

<OpenAIObject chat.completion id=chatcmpl-8RtpeGkrS00JVMslsPGVKRvLDohrS at 0x7f96202671a0> JSON: {
  "id": "chatcmpl-8RtpeGkrS00JVMslsPGVKRvLDohrS",
  "object": "chat.completion",
  "created": 1701660166,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Here are the 10 movies recommended for you: \n1. Twelve Monkeys (1995)\n2. Dead Man Walking (1995)\n3. Star Wars (1977)\n4. Fargo (1996)\n5. Godfather, The (1972)\n6. Return of the Jedi (1983)\n7. Jerry Maguire (1996)\n8. My Best Friend's Wedding (1997)\n9. Men in Black (1997)\n10. Contact (1997)"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 546,
    "completion_tokens": 100,
    "total_tokens": 646
  },
  "system_fingerprint": null
}

In [203]:
prediction_3 = response.choices[0].message.content
prediction_3

"Here are the 10 movies recommended for you: \n1. Twelve Monkeys (1995)\n2. Dead Man Walking (1995)\n3. Star Wars (1977)\n4. Fargo (1996)\n5. Godfather, The (1972)\n6. Return of the Jedi (1983)\n7. Jerry Maguire (1996)\n8. My Best Friend's Wedding (1997)\n9. Men in Black (1997)\n10. Contact (1997)"

In [206]:
# only get the movie name from prediction_3
recommendations = []
for movie in prediction_3.split('\n'):
    split_movie = movie.split('.')
    if len(split_movie) > 1:
        # Extract the movie title and strip any leading/trailing whitespaces
        recommendations.append(split_movie[1].strip())

recommendations

['Twelve Monkeys (1995)',
 'Dead Man Walking (1995)',
 'Star Wars (1977)',
 'Fargo (1996)',
 'Godfather, The (1972)',
 'Return of the Jedi (1983)',
 'Jerry Maguire (1996)',
 "My Best Friend's Wedding (1997)",
 'Men in Black (1997)',
 'Contact (1997)']

In [202]:
# hit rate of recommendations in watched movies
hit_rate = len(set(recommendations).intersection(set(watched_movies))) / len(recommendations)
hit_rate

0.5